In [162]:
import numpy as np
from scipy.sparse import diags
np.set_printoptions(precision=4)


In [163]:
#tridiagonal algorithm
def tridiagonal_algo(A, d):
    #matrix separation
    b = np.copy(np.diag(A))
    a = np.concatenate(([0], np.diagonal(A,-1))) #lower
    c = np.concatenate((np.diagonal(A,1), [0])) #upper
    n=len(d)
    x=np.zeros((n))
    #thomas algorithm
    #forward sweep:
    for i in range(1, n):
        w = a[i]/float(b[i-1])
        b[i] = b[i] - (w*c[i-1])
        d[i] = d[i] - (w*d[i-1])
    #back-substitution:
    x[n-1] = d[n-1]/b[n-1]
    for i in range(n-2, -1, -1):
        x[i] = (d[i] - c[i]*x[i+1])/b[i]
    return x

In [164]:
#testing tridiagonal algo
A=(np.array([[3,-1,0],
            [-1,3,-1],
            [0,-1,3]])).astype('float64')
d=(np.array([-1,7,7])).astype('float64')

print("x = ",tridiagonal_algo(A, d))

x =  [0.9524 3.8571 3.619 ]


In [165]:
#exact solution function
def u(x,t,f):
    if 0<=x<t:
        return 0
    elif t<=x<=1:
        return f(x-t)

#initial condition functions
def f_1(x):
    if x<0.5:
        return 0.25 - np.abs(x-0.25)
    else:
        return 0
f_2 = lambda x:np.power(np.sin(np.pi*x),2)

#definition of numerical error on max norm
e = lambda v_k, u_kn:np.amax(np.abs(np.subtract(v_k, u_kn)))

In [253]:
'''
M = 64
'''

#data initialization
b = 1
M = 64
h = T = 1/M
n = M
r = b*T/(2*h)
#Ax = B
A = diags([-r, 1, r], [-1, 0, 1], shape=(int(n+1), int(n+1))).toarray()
#fill B with kh
B = np.linspace(0, 1, n+1)

In [254]:
#to check the exact value (u(x,t)) per T time step
U = np.zeros(n+1)
for k in range(n+1):
    U[k] = u(k*h, 1/64, f_1)
print(U)

[0.     0.     0.0156 0.0312 0.0469 0.0625 0.0781 0.0938 0.1094 0.125
 0.1406 0.1562 0.1719 0.1875 0.2031 0.2188 0.2344 0.25   0.2344 0.2188
 0.2031 0.1875 0.1719 0.1562 0.1406 0.125  0.1094 0.0938 0.0781 0.0625
 0.0469 0.0312 0.0156 0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.    ]


In [266]:
'''
(i) initial condition data (f_1), the matrix pattern is Z[time][space], Z any 2D matrix
'''

V_mat = np.zeros((n+1,n+1)) #to save the overall result of v_k^n
f_1_vec = np.vectorize(f_1)
v = f_1_vec(B)
v[0] = 0
v[M] = 0
V_mat[0] = v

#do tridiagonal algo for numerical solution
for i in range(n): 
    v = tridiagonal_algo(A, v)
    V_mat[i+1] = v

#exact sol:
U = np.zeros((n+1, n+1))
for i in range(n+1):
    for k in range(n+1):
        U[i][k] = u(k*h, i*T, f_1)
        
#errors on max norm
errs = np.zeros(int(M/2)+1)
for i in range(int(M/2)+1):
    errs[i] = e(V_mat[i, 1:M], U[i, 1:M])
print(errs)

[0.     0.011  0.0166 0.0207 0.0242 0.0271 0.03   0.0321 0.0352 0.0363
 0.0393 0.0408 0.0427 0.0445 0.0462 0.0479 0.0494 0.051  0.0525 0.0539
 0.0553 0.0567 0.058  0.0594 0.0606 0.0619 0.0631 0.0643 0.0655 0.0666
 0.0677 0.0689 0.0698]


In [267]:
'''
(ii) initial condition data (f_2), the matrix pattern is Z[time][space], Z any 2D matrix
'''

V_mat = np.zeros((n+1,n+1)) #to save the overall result of v_k^n
f_2_vec = np.vectorize(f_2)
v = f_2_vec(B)
v[0] = 0
v[M] = 0
V_mat[0] = v

#do tridiagonal algo for numerical solution
for i in range(n): 
    v = tridiagonal_algo(A, v)
    V_mat[i+1] = v

#exact sol:
U = np.zeros((n+1, n+1))
for i in range(n+1):
    for k in range(n+1):
        U[i][k] = u(k*h, i*T, f_2)
        
#errors on max norm
errs = np.zeros(int(M/2)+1)
for i in range(int(M/2)+1):
    errs[i] = e(V_mat[i, 1:M], U[i, 1:M])
print(errs)

[0.     0.0024 0.0051 0.0087 0.0139 0.0215 0.0321 0.0461 0.0642 0.0906
 0.1211 0.1553 0.1929 0.2335 0.2767 0.3222 0.3693 0.4177 0.4669 0.5163
 0.5656 0.6141 0.6615 0.7072 0.7508 0.7919 0.83   0.8649 0.8961 0.9233
 0.9463 0.9648 0.9787]


In [268]:
'''
M = 128
'''
#data initialization
b = 1
M = 128
h = T = 1/M
n = M
r = b*T/(2*h)
#Ax = B
A = diags([-r, 1, r], [-1, 0, 1], shape=(int(n+1), int(n+1))).toarray()
#fill B with kh
B = np.linspace(0, 1, n+1)

In [269]:
'''
(i) initial condition data (f_1), the matrix pattern is Z[time][space], Z any 2D matrix
'''

V_mat = np.zeros((n+1,n+1)) #to save the overall result of v_k^n
f_1_vec = np.vectorize(f_1)
v = f_1_vec(B)
v[0] = 0
v[M] = 0
V_mat[0] = v

#do tridiagonal algo for numerical solution
for i in range(n): 
    v = tridiagonal_algo(A, v)
    V_mat[i+1] = v

#exact sol:
U = np.zeros((n+1, n+1))
for i in range(n+1):
    for k in range(n+1):
        U[i][k] = u(k*h, i*T, f_1)
        
#errors on max norm
errs = np.zeros(int(M/2)+1)
for i in range(int(M/2)+1):
    errs[i] = e(V_mat[i, 1:M], U[i, 1:M])
print(errs)

[0.     0.0055 0.0083 0.0104 0.0121 0.0136 0.015  0.0162 0.0174 0.0184
 0.0195 0.0204 0.0214 0.0222 0.0232 0.0238 0.0249 0.0253 0.0264 0.0269
 0.0277 0.0284 0.0291 0.0297 0.0304 0.031  0.0316 0.0322 0.0328 0.0334
 0.034  0.0346 0.0351 0.0357 0.0362 0.0367 0.0373 0.0378 0.0383 0.0388
 0.0393 0.0398 0.0403 0.0408 0.0412 0.0417 0.0422 0.0426 0.0431 0.0435
 0.044  0.0444 0.0448 0.0453 0.0457 0.0461 0.0465 0.047  0.0474 0.0478
 0.0482 0.0486 0.049  0.0494 0.0498]


In [270]:
'''
(ii) initial condition data (f_2), the matrix pattern is Z[time][space], Z any 2D matrix
'''

V_mat = np.zeros((n+1,n+1)) #to save the overall result of v_k^n
f_2_vec = np.vectorize(f_2)
v = f_2_vec(B)
v[0] = 0
v[M] = 0
V_mat[0] = v

#do tridiagonal algo for numerical solution
for i in range(n): 
    v = tridiagonal_algo(A, v)
    V_mat[i+1] = v

#exact sol:
U = np.zeros((n+1, n+1))
for i in range(n+1):
    for k in range(n+1):
        U[i][k] = u(k*h, i*T, f_2)
        
#errors on max norm
errs = np.zeros(int(M/2)+1)
for i in range(int(M/2)+1):
    errs[i] = e(V_mat[i, 1:M], U[i, 1:M])
print(errs)

[0.0000e+00 6.3270e-04 1.3848e-03 2.4127e-03 3.9206e-03 6.1191e-03
 9.1880e-03 1.3255e-02 1.8394e-02 2.4634e-02 3.1972e-02 4.0388e-02
 4.9856e-02 6.0345e-02 7.2175e-02 8.5443e-02 9.9718e-02 1.1497e-01
 1.3115e-01 1.4823e-01 1.6616e-01 1.8490e-01 2.0441e-01 2.2463e-01
 2.4553e-01 2.6704e-01 2.8911e-01 3.1170e-01 3.3474e-01 3.5818e-01
 3.8197e-01 4.0604e-01 4.3034e-01 4.5480e-01 4.7938e-01 5.0400e-01
 5.2861e-01 5.5315e-01 5.7756e-01 6.0178e-01 6.2575e-01 6.4941e-01
 6.7271e-01 6.9559e-01 7.1799e-01 7.3985e-01 7.6114e-01 7.8179e-01
 8.0175e-01 8.2097e-01 8.3942e-01 8.5704e-01 8.7379e-01 8.8963e-01
 9.0452e-01 9.1843e-01 9.3132e-01 9.4316e-01 9.5392e-01 9.6358e-01
 9.7212e-01 9.7950e-01 9.8572e-01 9.9076e-01 9.9460e-01]
